### **3.2 - Properly fetch your Variables**

¿Sabías que tu DAG se analiza cada treinta segundos por defecto, tal y como se define en el parámetro "min_file_process_interval", y esto puede ser un gran problema para ti si tienes muchas variables y no estás obteniendo tus variables correctamente? Déjame mostrarte exactamente lo que quiero decir. En tu data pipeline, para obtener la variable "my_dag_partner_secret" necesitas usar 'Variable.get' y luego sacas la variable correspondiente de la meta database de Airflow, así que para hacer eso creas una conexión. Pero aquí está la cosa, si usted obtiene la misma variable fuera de la función "_extract", por ejemplo, aquí o incluso aquí:

<center><img src="https://i.postimg.cc/zXT2dDNn/a1114.png"></center>

vas a crear una conexión cada vez que tu DAG sea analizado. Cada 30 segundos vas a crear una conexión inútil para obtener la variable, incluso si no la usas en este momento, incluso si tu DAG no se está ejecutando todavía. Y esto es un gran problema porque si tienes muchos DAGs y manejas muchas variables vas a crear una tonelada de conexiones inútiles a la meta database y tu base de datos podría terminar con algunos problemas. **`Así que tenga en cuenta que como una mejor práctica siempre debe tratar en lo posible para obtener sus variables dentro de sus tareas o mediante el uso del templates engine como vas a ver más adelante`**. A veces tienes que hacerlo así, por ejemplo, si generas algunas tareas basadas en el valor de la variable entonces en ese caso necesitas hacerlo pero por favor trata de evitar esto porque creas conexiones inútiles cada 30 segundos por defecto. 

Dicho esto, hay algunas cosas que hablar acerca de las variables y la primera es lo que si usted tiene algunas variables que son realmente [???] entre sí, por ejemplo, digamos que usted tiene el nombre de su partner y luego el password de su partner y tal vez el path de su partner, así que tienes esas tres variables que están realmente relacionadas entre sí. ¿Vas a llamar a "Variable.get" tres veces?. No, no lo vas a hacer, porque, vas a crear como tres conexiones sólo para obtener el mismo tipo de información al final. Así que para resolver esto y sólo hacer una conexión a la meta database, puedes fusionar (merge) todas tus variables en una sola variable especificando el valor JSON. Veamos cómo hacerlo. En la interfaz de Airflow vamos a modificar la variable "my_dag_partner_secret"

<center><img src="https://i.postimg.cc/NG9zH7PC/a1115.png"></center>

y en "Val" vamos a definir un valor JSON:

<center><img src="https://i.postimg.cc/pLxG3NV3/a1116.png"></center>

Así que como puedes ver aquí tienes tres valores diferentes en una sola variable y por lo tanto vas a crear una sola conexión para obtener todos esos valores. Haz clic en "Save" y como podemos ver el valor de tu variable es el siguiente valor Json:

<center><img src="https://i.postimg.cc/3xBSTLdN/a1117.png"></center>

Así que ahora vamos a ver cómo obtenerlo en tu DAG. Modifiquemos el nombre de la variable, llamémosla "partner_settings" y aquí lo único que tienes que añadir es "deserialize_json=True" y entonces vas a obtener el valor JSON como un diccionario y así podrás acceder a los diferentes valores. 

<center><img src="https://i.postimg.cc/d0VWt8yB/a1118.png"></center>

Así que, por ejemplo, si quieres obtener el nombre de tu partner, sólo tienes que escribir:

<center><img src="https://i.postimg.cc/DzpxW8QK/a1119.png"></center>

Y quieres obtener la "api_key" y la "path":

<center><img src="https://i.postimg.cc/RF3Xw4GJ/a1120.png"></center>

Y es tan sencillo como eso y al hacer esto sólo creas una conexión en lugar de crear tres conexiones. Así que no dudes en poner el valor JSON en tu variable si tienes varias variables pero que estén realmente relacionadas entre sí y eso no sólo hará menos conexiones a tu meta database sino que también hará que tu DAG sea más limpio. No olvides modificar el nombre de tu variable a:

<center><img src="https://i.postimg.cc/nz6TkgwR/a1121.png"></center>

Estás listo para ver el último punto importante sobre las variables. Esto podría sorprenderte. Digamos que quieres pasar el nombre de tu partner como parámetro de la función de Python "_extract", ya sabes que para hacerlo, pones digamos "partner_name" y luego como usas el PythonOperator necesitas usar el argumento "op_args" o "op_kwargs". Usemos "op_args" igual a una Lista. Necesitas poner el valor de "partner_name". Así que en ese caso quieres obtener el nombre de tu partner de tu variable, de la variable "my_dag_partner", así que para hacer eso puedes copiar esta línea y pegarla dentro de los paréntesis de la Lista:

<center><img src="https://i.postimg.cc/Wzg8f8B2/a1122.png"></center>
<center><img src="https://i.postimg.cc/1RqvtRfL/a1123.png"></center>

¿Qué pasa si haces eso? Bueno, vas a hacer una conexión cada vez que tu DAG es analizado, aunque puedas pensar que no es el caso porque cuando obtienes tu variable en la lista usando el argumento "op_args", pero, es el caso, si haces eso vas a obtener esa variable:

<center><img src="https://i.postimg.cc/wTY0sKN3/a1124.png"></center>

cada vez que su DAG es analizado, cada 30 segundos por defecto. Pero hay una manera de arreglar esto, hay otra manera para evitar obtener su variable cada vez que su DAG es analizado. Y esta forma es aprovechando el "Template engine". Así que, básicamente, en lugar de obtener tus variables utilizando "Variable.get" vas a utilizar:

<center><img src="https://i.postimg.cc/dtRn2WR6/a1125.png"></center>

Y al hacer esto tu variable, aquí, será obtenida sólo una vez que tu DAG se esté ejecutando. Y esta es la potencia de la "template jinja" así que aquí se evita hacer la conexión cada vez que su DAG es analizado. Y también puedes hacer eso con el BashOperator, si tienes el BashOperator y un bash_command y quieres obtener la variable, puedes usar el "template engine". Básicamente puedes usar esa notación cada vez que los argumentos con los que estás tratando sean templated.